In [22]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
import html
import ast
import time
import pprint
import pandas as pd
import datetime
import csv
from bs4 import BeautifulSoup
url = "https://www.canberrahealthservices.act.gov.au/_designs/generic-modules/entitywaitingtimes/asset-listings/dynamic-wait-times-data-raw-asset-listing/_nocache"

In [4]:
curr_time = str(datetime.datetime.now())
filename = 'chs_wait_times_'+curr_time+'.csv'

In [20]:
# Set up Google Sheets API credentials
key = 'MESSAGE_ME_FOR_MY_KEY'
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(key, scope)
client = gspread.authorize(creds)

# Open the Google Sheet (use the name or URL of your sheet)
spreadsheet = client.open('wic_wait_times')
worksheet = spreadsheet.get_worksheet(0)  # Open the first sheet

In [21]:
header_written = False

try:
    while True:
        # Get current wait times as dictionary
        response = requests.get(url)  # HTTP request
        soup = BeautifulSoup(response.text, 'html.parser')  # Create parser
        data_element = soup.find('div', id='divEntitiesJSON').get_text(strip=True)  # Get element from page
        to_dict = ast.literal_eval(data_element)  # Convert to dictionary
        
        # Convert to single data row
        rows = []
        for entry in to_dict:
            id_ = entry['id']
            last_updated = entry['lastUpdated']
            clinic_type = entry['type']
            queue = next((x['queue'] for x in entry['waitTimes'] if x['name'] == 'Wait'), None)
            wait_time = next((x['duration'] for x in entry['waitTimes'] if x['name'] == 'Wait'), None)
            treat_time = next((x['duration'] for x in entry['waitTimes'] if x['name'] == 'Treat'), None)
            total_time = next((x['duration'] for x in entry['waitTimes'] if x['name'] == 'Total'), None)
            to_append = {
                'id': id_,
                'lastUpdated': last_updated,
                'clinic_type': clinic_type,
                'queue': queue,
                'wait_time': wait_time,
                'treat_time': treat_time,
                'total_time': total_time
            }
            rows.append(to_append)
        
        # Convert rows to DataFrame
        df = pd.DataFrame(rows)
        
        # Append DataFrame to Google Sheet
        if not header_written:
            worksheet.append_row(df.columns.tolist())  # Write headers to Google Sheet
            header_written = True
        
        for row in df.itertuples(index=False, name=None):
            worksheet.append_row(list(row))  # Append each row to Google Sheet
        print("Successfully printed stats from ACT Health! Zzzz...")
        time.sleep(59)
except Exception as e:
    print(str(datetime.datetime.now()))
    raise e

Successfully printed stats from ACT Health! Zzzz...


KeyboardInterrupt: 